In [ ]:
# pip install fastapi uvicorn[standard]

# 0. Базовое FastAPI-приложение

## 1. Введение

FastAPI — это современный фреймворк для создания веб-сервисов на Python, разработанный с упором на **высокую производительность**, **типобезопасность** и **автоматическую генерацию документации** по спецификации OpenAPI.

Он используется в MLOps-практике как интерфейсный слой, через который модели машинного обучения становятся доступными внешним системам — например, через HTTP-запросы или REST API.

## 2. Инициализация проекта

Каждое приложение FastAPI представляет собой обычный Python-пакет, внутри которого создаётся экземпляр класса `FastAPI`. Этот объект и есть точка входа в веб-приложение.
Для удобства проект оформляется в виде пакета `app`, где файл `main.py` содержит минимальный код инициализации.

Пример структуры:

```
fastapi-practice/
 ├── app/
 │    └── main.py
 └── .venv/
```

## 3. Минимальный пример кода

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello World"}
```

В этом примере:

* `app = FastAPI()` — создаёт объект приложения;
* декоратор `@app.get("/")` регистрирует HTTP-маршрут `GET /`;
* асинхронная функция-обработчик (`async def`) возвращает словарь, который FastAPI автоматически сериализует в **JSON**-ответ;
* документация и схема API генерируются автоматически на основе аннотаций типов и сигнатур функций.

## 4. Запуск и проверка

Сервер запускается командой:

```bash
uvicorn app.main:app --reload
```

* `app.main:app` — путь к объекту `app` внутри модуля `app/main.py`;
* `--reload` включает автоматическую перезагрузку при изменениях кода (для разработки).

После запуска:

* Главная страница доступна по адресу `http://127.0.0.1:8000/`;
* Автоматическая документация — по адресу
  **Swagger UI:** `http://127.0.0.1:8000/docs`;
  **OpenAPI JSON:** `http://127.0.0.1:8000/openapi.json`.

При обращении к `GET /` сервер вернёт:

```json
{"message": "Hello World"}
```

# 1. Параметры маршрутов (path и query)

## 1. Общая идея

Любое API-приложение должно принимать входные данные. В FastAPI существует два базовых типа параметров:

Path параметры — часть пути (например, `/items/42`), обычно обозначают уникальные ресурсы.

Query параметры — часть строки после знака ? (например, `/search?q=cat&limit=5`), описывают фильтры, сортировки, лимиты и т. п.

FastAPI автоматически валидаирует типы параметров на основе аннотаций Python и добавляет их описание в документацию Swagger.

### Path-параметры (параметры пути)

Path — это часть пути запроса, прямо встроенная в сам адрес (в "тело" URL).
Она описывает конкретный ресурс или объект, к которому мы обращаемся.

In [ ]:
@app.get("/items/{item_id}")
async def get_item(item_id: int):
    return {"item_id": item_id}

FastAPI автоматически достаёт `item_id` из адреса (`/items/42`) и подставляет его в аргумент функции.

Если пользователь введёт `/items/abc`, а тип задан как `int`, — FastAPI вернёт ошибку `422` (валидация типов).

### Query-параметры (параметры запроса)

Query — это дополнительные параметры, которые идут после знака `?` в URL.

Они не указывают на конкретный ресурс, а уточняют, как или что именно нужно получить.

In [ ]:
@app.get("/search")
async def search_items(q: str, limit: int = 10):
    return {"query": q, "limit": limit}

Query-параметры:

* всегда передаются после ?;
* могут быть в любом порядке;
* могут быть необязательными;
* можно передавать несколько сразу (?page=2&sort=asc).

`GET /users/15/history?limit=3&order=desc`

`Path: /users/15/history`

`Querry: ?limit=3&order=desc`

In [ ]:
from fastapi import FastAPI, Query, Path

@app.get("/users/{user_id}/items")
async def get_user_items(
    user_id: int = Path(..., description="ID пользователя"),
    category: str | None = Query(None, description="Фильтр по категории"),
    limit: int = Query(10, ge=1, le=100, description="Максимальное число элементов")
):
    pass

## 2. Что такое роутер в FastAPI

Роутер (Router) — это модуль, который группирует несколько эндпоинтов (маршрутов), обычно относящихся к одной логической части приложения.

`APIRouter` — это способ вынести маршруты в отдельные файлы и потом подключить их в основное приложение.

```
app/
 ├── main.py
 └── api/
     ├── users.py
     └── items.py
```

In [ ]:
# app/api/users.py
from fastapi import APIRouter

router = APIRouter(prefix="/users", tags=["Users"])

@router.get("/")
def list_users():
    return [{"id": 1, "name": "Alice"}]

@router.get("/{user_id}")
def get_user(user_id: int):
    return {"id": user_id, "name": f"User {user_id}"}

In [ ]:
# app/api/items.py
from fastapi import APIRouter

router = APIRouter(prefix="/items", tags=["Items"])

@router.get("/")
def list_items():
    return [{"id": 1, "title": "Book"}]


In [ ]:
# app/main.py

from fastapi import FastAPI
from app.api import users, items

app = FastAPI(title="Modular FastAPI Example")

app.include_router(users.router)
app.include_router(items.router)


## 3. Pydantic BaseModel

BaseModel — это базовый класс Pydantic, который описывает структуру, типы и ограничения данных.

Он используется для проверки входных данных и формирования корректных ответов API.

Каждая модель на основе BaseModel:

* проверяет типы данных (int, str, float и т. д.);
* применяет валидацию (например, min_length, ge, EmailStr);
* автоматически документируется в Swagger UI;
* гарантирует, что сервис не примет и не вернёт “грязные” данные.

```
app/
 ├── main.py
 ├── api/
 │    ├── users.py
 │    └── items.py
 └── models/
      └── schemas.py
 ```

```py
class ItemBase(BaseModel):
    """Базовые поля, общие для всех схем предмета."""
    title: str = Field(..., min_length=1, max_length=100, description="Название предмета")
    description: Optional[str] = Field(None, max_length=300, description="Описание предмета")
```

## Перечень типовых сценариев взаимодействия FastAPI в ML-сервисах:

1. **Загрузка изображения (единичный инференс)**

   *Вход:* `UploadFile` (тип multipart/form-data).

   *Выход:* `JSONResponse` при возврате предсказаний (метки, вероятности) либо `Response` с `media_type="image/png|jpeg"` при возврате визуализированного результата.

   `UploadFile` обеспечивает потоковую обработку и минимальное использование памяти; multipart является стандартом для клиентских SDK и браузеров.

2. **Потоковое видео (одиночный источник)**

   *Вход:* строковый параметр с URI (`rtsp://`, `http://`) или `UploadFile` при передаче файла.

   *Выход:* `StreamingResponse` с генератором кадров, `media_type="multipart/x-mixed-replace"` (MJPEG) либо `video/mp4` при упаковке потока.
   
   Обеспечивает последовательную передачу кадров без полной буферизации и минимальную задержку.

3. **NumPy-массивы и числовые данные (классические ML-модели)**
   
   *Вход:*
   — `BaseModel` с полями `list[float]` или `list[list[float]]` при работе с малым объёмом данных (JSON);
   — `UploadFile`/`bytes` с бинарным содержимым `.npy`/`.npz` при больших массивах.
   
   *Выход:*
   — `FileResponse`/`Response`
   
   JSON-сериализация удобна, но неэффективна для крупных матриц; бинарные форматы сохраняют точность и ускоряют ввод-вывод.

4. **Обработка текста (NLP-модели, классификация, генерация)**
   
   *Вход:* `BaseModel` или `Form(...)` со строковыми полями.
   
   *Выход:* `JSONResponse` при возврате токенов, меток, вероятностей или `StreamingResponse` при поэтапной генерации текста (например, для чат-моделей).
   
   Текстовые данные естественно сериализуются в JSON; при генерации в реальном времени применяется потоковая передача фрагментов вывода.

В обобщённом виде:
`UploadFile` предназначен для бинарных данных (изображения, аудио, видео)

`BaseModel` — для структурированных текстовых и числовых данных

`StreamingResponse` и `WebSocket` — для потоковых сценариев

## Самостоятельная работа

Разработать **ML-сервис на FastAPI**, который:

1. Принимает **изображение** в запросе (через `UploadFile`).
2. Выполняет **обработку изображения** с помощью модели (детекция или сегментация).
3. Возвращает **изображение с нанесёнными предсказаниями** (bounding boxes, маски и т.п.) в виде ответа (`StreamingResponse` или `FileResponse`).

**Требования:**

* Валидация входных данных.
* Автоматическая документация доступна в `/docs`.
* Результат работы можно протестировать напрямую через Swagger UI.

**Дополнительно (по желанию):**

* Добавить логирование времени инференса.
* Вернуть вместе с изображением JSON с координатами детекций.
